In [1]:
from datetime import datetime
from portfolio.equity_portfolio import EquityPortfolio
from analyzer.analyzer import Analyzer as a
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
start = datetime(2017,1,1)
end = datetime(2020,1,1)

In [3]:
ep = EquityPortfolio(start,end,"portfolio",seats=10,strats=
                    {
                        "rolling_percent":{"params":{"timeframe":"daily"
                ,"requirement":5
                ,"days":100
                ,"value":False
                ,"currency":"prices"}},
                            "progress_report":{"params":{"timeframe":"quarterly"
                    ,"requirement":5}},
                    "financial_predict":{"params":{}},
                      "speculation":{"params":{}},})

In [4]:
ep.strats

{'rolling_percent': {'params': {'timeframe': 'daily',
   'requirement': 5,
   'days': 100,
   'value': False,
   'currency': 'prices'}},
 'progress_report': {'params': {'timeframe': 'quarterly', 'requirement': 5}},
 'financial_predict': {'params': {}},
 'speculation': {'params': {}}}

In [5]:
ep.load()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:46<00:00, 11.71s/it]


In [6]:
ep.transform()

 50%|███████████████████████████████████████████████████████████████████▌                                                                   | 2/4 [00:00<00:00, 10.64it/s]


AttributeError: 'NoneType' object has no attribute 'drop'

In [11]:
ep.strats["rolling_percent"]["class"].subscriptions

{'market': {'preload': True,
  'tables': {'prices': Empty DataFrame
   Columns: []
   Index: [0]},
  'db': <database.market.Market at 0x2403af5ea10>}}

In [ ]:
ep.sim()

In [ ]:
trades = ep.backtest()

In [ ]:
trades["strategy"].unique()

In [ ]:
pva = a.pv_analysis(ep)
iva = a.industry_analysis(ep)

In [ ]:
plt.plot(pva["date"],pva["progress_report"])
plt.plot(pva["date"],pva["rolling_percent"])
plt.plot(pva["date"],pva["financial_predict"])
plt.plot(pva["date"],pva["speculation"])
plt.legend([x for x in pva.columns if x != "date"])

In [ ]:
ep.strats["speculation"]["class"].ai